# Problema de PCP






### Importando as Bibliotecas

Nesta atividade serão utilizadas as bibliotecas:

    numpy para manipulação de vetores

    pandas para manipuação de tabelas

    ortools para modelagem.

ORTools é uma biblioteca muito versátil, ela aceita manipulação de vetores para a criação de modelos, já instalar os mais populares solvers de código aberto e comerciais, ela é toda construida em C++ otimizado, resultando em uma modelagem rápida, computacionalmente falando, por mais que essa seja a parte que menos consume memória em problemas de pesquisa operacional, ela garante uma performance semelhante mesmo sendo executada em linguagem interpretadas ou compiladas.

In [ ]:
""" 
Vinicius Costa Gandolfi
"""

!pip install ortools
import numpy as np # Manipulação de Vetores
import pandas as pd # Manipulação de  Tabelas
from ortools.linear_solver import pywraplp # Biblioteca de modelagem de pesquisa operaciona



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.0 MB 8.4 MB/s 
     |████████████████████████████████| 408 kB 45.0 MB/s 
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.17.3
    Uninstalling protobuf-3.17.3:
      Successfully uninstalled protobuf-3.17.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.7 which is incompatible.
tensorflow-metadata 1.10.0 requires protobuf<4,>=3.13, but you have protobuf 4.21.7 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.21.7 which is incompatible.
google-cloud-bigquery-storage 1.1.2 requires protobuf<4.0.0dev, but you have protobuf 4.21.7 which is incompatible.
google-api-core 1.31.

In [ ]:
produtos = pd.DataFrame({
       "Produto"        : [1, 2, 3, 4, 5, 6],
       "Etapa1: Máquina": [1, 4, 1, 4, 1, 2],
       "Etapa1: Tempo"  : [1, 2, 3, 1, 2, 3],
       "Etapa2: Máquina": [2, 2, 3, 3, 4, 3],
       "Etapa2: Tempo"  : [3, 2, 1, 3, 2, 1],
       "Lucro"          : [5, 2, 6, 3, 7, 4]
       })

produtos

,Produto,Etapa1: Máquina,Etapa1: Tempo,Etapa2: Máquina,Etapa2: Tempo,Lucro
0,1,1,1,2,3,5
1,2,4,2,2,2,2
2,3,1,3,3,1,6
3,4,4,1,3,3,3
4,5,1,2,4,2,7
5,6,2,3,3,1,4


In [ ]:
def df_to_matrix(df, columns_of_steps, columns_of_time):
  dummies = []
  column_index = np.arange(len(columns_of_steps))

  if len(columns_of_steps) != len(columns_of_time):
    return print('Ops... O número de colunas de etapas devem ser o mesmo número de colunas de tempo.')

  for index in column_index:
    dummie = pd.get_dummies(df[columns_of_steps[index]], prefix='Maquina')
    dummie.values[dummie != 0] = df[columns_of_time[index]]
    dummies.append(dummie)

  resp = pd.concat(dummies)
  resp = resp.sum(level=0)
  resp = resp.transpose()
  resp = resp.add_prefix('Produto_')

  return resp

In [ ]:
restMaquinas = df_to_matrix(produtos, ['Etapa1: Máquina', 'Etapa2: Máquina'], 
             ['Etapa1: Tempo', 'Etapa2: Tempo'])
restMaquinas

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


,Produto_0,Produto_1,Produto_2,Produto_3,Produto_4,Produto_5
Maquina_1,1.0,0.0,3.0,0.0,2.0,0.0
Maquina_2,3.0,2.0,0.0,0.0,0.0,3.0
Maquina_4,0.0,2.0,0.0,1.0,2.0,0.0
Maquina_3,0.0,0.0,1.0,3.0,0.0,1.0


In [ ]:
disponibilidade = pd.DataFrame({
  "Máquina"          : [ 1, 2, 3, 4],
  "Tempo Disponível" : [ 9, 12, 13, 9],
})

disponibilidade

,Máquina,Tempo Disponível
0,1,9
1,2,12
2,3,13
3,4,9


## PCP

Maximize $\sum_{j=1}^n$ $x_{j}$ * $l_{j}$ 

sujeito a:


$\sum_{i=1}^n$ $\sum_{j=1}^n$ $x_{j}$ * $T_{i, j}$ ≤ D  

 $x_{j}$ ≤  $y_{j}$ * M

 $\sum_{j=1}^n$ $y_{j}$ ≤ K

onde:

x = peças a serem produzidas de cada peça j

l = lucro por peça j

T = matriz com as restrições de tempo consumido por peça j por maquina i

D = tempo disponível por cada máquina i

M = maior tempo disponível

k = número máximo de peças diferentes produzidas


$x_{j}$ ∈ $Z^+$  ∀ j = 1... n

$y_j$ = {0, 1} ∀ j = 1... n

In [ ]:
# Inicializa o problema p que será resolvido com o solver 
solver = pywraplp.Solver("PCP", pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)

## Definindo $x_{j}$ como números Inteiros
***
O $x_{j}$ pertence aos números inteiros positivos. Pois cada produto deve ser feito de forma individua, ou seja, não exisste possibilidade de fazer 0.5 produto.


R1: $x_{j}$ ∈ $Z^+$  ∀ j = 1... n

In [ ]:
x = {} # Definindo x como um dicionário
for j in range(len(produtos['Produto'])):
  x[j] = solver.IntVar(0, solver.infinity(), f"x[{j}]")

print('Numero Variáveis: ', solver.NumVariables())

Numero Variáveis:  6


## Definindo $y_{j}$ Binário
***
O $y_{j}$ pertence aos binários. Pois para a restrição de produção de até 4 produtos é necessária uma variável binária.


R1: $y_{j}$ ∈ $Z^+$  ∀ j = 1... n

In [ ]:
y = {}
for j in range(len(produtos['Produto'])):
  y[j] = solver.IntVar(0, 1, f"y[{j}]")

print('Numero Variáveis: ', solver.NumVariables())

Numero Variáveis:  12


##Restrição 01
***

Cada peça tem que passar em duas máquinas, e esta matriz $T_{i, j}$ mostra qual produto passa em qua máquina, e o tempo para cada processo.
O D representa a quantidade disponível de tempo por maquina por dia.


$\sum_{i=1}^n$ $\sum_{j=1}^n$ $x_{j}$ * $T_{i, j}$ ≤ D  

In [ ]:
restMaquinasArray = restMaquinas.to_numpy()
dispMaquinasArray = disponibilidade['Tempo Disponível'].to_numpy()

rangeI = range(len(restMaquinasArray))
rangeJ = range(len(restMaquinasArray[0]))

for i in rangeI:
  solver.Add(solver.Sum([x[j] * restMaquinasArray[i, j]   for j in rangeJ]) <= dispMaquinasArray[i], name='RestMaquinas '+str(i))


print('Numero Restrições: ', solver.NumConstraints())

Numero Restrições:  4


 ## Restrição 02
***
Como a empresa tem de escolher entre tres ou menos produtos para a sua linha de produção, esta restrição vincura q quantidade de cada produto a uma varíavel binária, obrigando o modelo escolher se vai ou não produzir aquele produto.
 
 $x_{j}$ ≤  $y_{j}$ * M


In [ ]:
numProdutos = np.arange(len(restMaquinasArray[0]))
M = np.max(disponibilidade['Tempo Disponível'].to_numpy())

for j in numProdutos:
  solver.Add(x[j] <= y[j]*M, name='RestPeça '+str(j))

print('Numero Restrições: ', solver.NumConstraints())

Numero Restrições:  10


 ## Restrição 03
***
No final, a restrição 02 serve para conseguirmos definir, aqui na restrição 03 o número máximo de produtos escolhidos para a produção.
 
 $\sum_{j=1}^n$ $y_{j}$ ≤ K


In [ ]:
numProdutos = np.arange(len(restMaquinasArray[0]))
solver.Add(solver.Sum([numProdutos[j] * y[j]   for j in rangeJ]) <= 3, name='RestNumPeças'+str(i))

print('Numero Restrições: ', solver.NumConstraints())

Numero Restrições:  11


## Função de Objetivo
***

O objetivo deste problema é máximizar o lucro na produção de peças.

Maximize $\sum_{j=1}^n$ $x_{j}$ * $l_{j}$ 

In [ ]:
funcaoObjetivo = []
preçoProdutos = produtos['Lucro'].to_numpy()

for j in rangeJ:
  funcaoObjetivo.append(x[j] * preçoProdutos[j])

solver.Maximize(solver.Sum(funcaoObjetivo))
status = solver.Solve()
status

0

In [ ]:
print(solver.ExportModelAsLpFormat(obfuscated=False))

\ Generated by MPModelProtoExporter
\   Name             : PCP
\   Format           : Free
\   Constraints      : 11
\   Variables        : 12
\     Binary         : 6
\     Integer        : 6
\     Continuous     : 0
Maximize
 Obj: +5 x[0] +2 x[1] +6 x[2] +3 x[3] +7 x[4] +4 x[5] 
Subject to
 RestMaquinas_0: +1 x[0] +3 x[2] +2 x[4]  <= 9
 RestMaquinas_1: +3 x[0] +2 x[1] +3 x[5]  <= 12
 RestMaquinas_2: +2 x[1] +1 x[3] +2 x[4]  <= 13
 RestMaquinas_3: +1 x[2] +3 x[3] +1 x[5]  <= 9
 RestPeça_0: +1 x[0] -13 y[0]  <= 0
 RestPeça_1: +1 x[1] -13 y[1]  <= 0
 RestPeça_2: +1 x[2] -13 y[2]  <= 0
 RestPeça_3: +1 x[3] -13 y[3]  <= 0
 RestPeça_4: +1 x[4] -13 y[4]  <= 0
 RestPeça_5: +1 x[5] -13 y[5]  <= 0
 RestNumPeças3: +1 y[1] +2 y[2] +3 y[3] +4 y[4] +5 y[5]  <= 3
Bounds
 0 <= x[0] <= inf
 0 <= x[1] <= inf
 0 <= x[2] <= inf
 0 <= x[3] <= inf
 0 <= x[4] <= inf
 0 <= x[5] <= inf
 0 <= y[0] <= 1
 0 <= y[1] <= 1
 0 <= y[2] <= 1
 0 <= y[3] <= 1
 0 <= y[4] <= 1
 0 <= y[5] <= 1
Binaries
 y[0]
 y[1]
 y[2]
 

In [ ]:
for constraint in solver.constraints():
  print(constraint.dual_value())

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


In [ ]:
print('Resultado do PCP \n')

print('Função Objetivo: ', solver.Objective().Value())
for j in rangeJ:
  if x[j].solution_value() != 0:
    print(f"{x[j].solution_value()} unidade do produto {j+1}")
print('Tempo para solucionar: ', solver.wall_time(), ' milesegundos.')
print('Problem solved in ', solver.iterations(), ' iterações.')

Resultado do PCP 

Função Objetivo:  30.000000000000004
6.0 unidade do produto 2
3.0 unidade do produto 3
Tempo para solucionar:  168  milesegundos.
Problem solved in  7  iterações.


## Conclusão e Generalização.

Como conseguimos chegar no modelo genérico para resover este problema, podemos o coocar em uma função e generalizarmos para qualquer problema similar, apenas mudando os parametros.

In [ ]:
class PCP:
  def __init__(self, matrizMaquinaProduto, matrizDisponibilidade, precoProdutos, numeroDeProdutosPermitidos=None):
    self.matrizMaquinaProduto = matrizMaquinaProduto
    self.matrizDisponibilidade = matrizDisponibilidade
    self.numeroDeProdutosPermitidos = numeroDeProdutosPermitidos
    self.precoProdutos = precoProdutos
    self.solver = pywraplp.Solver("PCP", pywraplp.Solver.SCIP_MIXED_INTEGER_PROGRAMMING)
    self.status = None

  def solve(self):

    rangeJ = np.arange(len(self.matrizMaquinaProduto[0]))
    rangeI = np.arange(len(self.matrizMaquinaProduto))

    x = {} 
    # Definindo variável inteira X
    for j in rangeJ:
      x[j] = self.solver.IntVar(0, self.solver.infinity(), f"x[{j}]")
    

    # Restrição 1
    for i in rangeI:
      self.solver.Add(self.solver.Sum([x[j] * self.matrizMaquinaProduto[i, j]   for j in rangeJ]) <= self.matrizDisponibilidade[i], name='RestMaquinas '+str(i))
    

    if self.numeroDeProdutosPermitidos:
      y = {}
      # Definindo variável binária Y
      for j in rangeJ:
        y[j] = self.solver.IntVar(0, 1, f"y[{j}]")
      

      # Restrição 2
      for j in numProdutos:
        self.solver.Add(x[j] <= y[j]*M, name='RestPeça '+str(j))
      
      
      # Restrição 3
      self.solver.Add(self.solver.Sum([numProdutos[j] * y[j]   for j in rangeJ]) <= 3, name='RestNumPeças'+str(i))
      
    
    # Objetivo
    funcaoObjetivo = []

    for j in rangeJ:
      funcaoObjetivo.append(x[j] * self.precoProdutos[j])

    self.solver.Maximize(solver.Sum(funcaoObjetivo))
    self.status = self.solver.Solve()

    return self.status

  def imprimirModelo(self):
    if self.status == 0:
      return print(self.solver.ExportModelAsLpFormat(obfuscated=False))

  def imprimirResultado(self):

    if self.status == 0:
      print('Resultado do PCP')
      print('Função Objetivo: ', self.solver.Objective().Value())
      print('Tempo para solucionar: ', self.solver.wall_time(), ' milesegundos.')
      print('Problem solved in ', self.solver.iterations(), ' iterações.')

      for j in rangeJ:
        
        if x[j].solution_value() != 0:
          print(f"{int(x[j].solution_value())} unidade do produto {j+1}")

In [ ]:
matrizMaquinaProduto = restMaquinas.to_numpy()
matrizDisponibilidade = disponibilidade['Tempo Disponível'].to_numpy()
precoProdutos = produtos['Lucro'].to_numpy()
numeroDeProdutosPermitidos = 3

pcp = PCP(matrizMaquinaProduto, matrizDisponibilidade, precoProdutos, numeroDeProdutosPermitidos)
pcp.solve()
pcp.imprimirResultado()
print('-'*100)
pcp.imprimirModelo()

Resultado do PCP
Função Objetivo:  30.000000000000004
Tempo para solucionar:  8  milesegundos.
Problem solved in  7  iterações.
6 unidade do produto 2
3 unidade do produto 3
----------------------------------------------------------------------------------------------------
\ Generated by MPModelProtoExporter
\   Name             : PCP
\   Format           : Free
\   Constraints      : 11
\   Variables        : 12
\     Binary         : 6
\     Integer        : 6
\     Continuous     : 0
Maximize
 Obj: +5 x[0] +2 x[1] +6 x[2] +3 x[3] +7 x[4] +4 x[5] 
Subject to
 RestMaquinas_0: +1 x[0] +3 x[2] +2 x[4]  <= 9
 RestMaquinas_1: +3 x[0] +2 x[1] +3 x[5]  <= 12
 RestMaquinas_2: +2 x[1] +1 x[3] +2 x[4]  <= 13
 RestMaquinas_3: +1 x[2] +3 x[3] +1 x[5]  <= 9
 RestPeça_0: +1 x[0] -13 y[0]  <= 0
 RestPeça_1: +1 x[1] -13 y[1]  <= 0
 RestPeça_2: +1 x[2] -13 y[2]  <= 0
 RestPeça_3: +1 x[3] -13 y[3]  <= 0
 RestPeça_4: +1 x[4] -13 y[4]  <= 0
 RestPeça_5: +1 x[5] -13 y[5]  <= 0
 RestNumPeças3: +1 y[1] +2

Questão 02

***

In [ ]:
def stringToDict(string, string2, string3):
  dicionario = {
        "Produto"        : [],
        "Etapa1: Máquina": [],
        "Etapa1: Tempo"  : [],
        "Etapa2: Máquina": [],
        "Etapa2: Tempo"  : [],
        "Lucro"          : []
        }

  arrays = np.array([string.split('\n')])
  arrays = arrays.T
  t = 0
  for array in arrays:
    if t >=2:
      array = np.array(array[0].split(' ')).transpose()
      c = 0
      for arr in array:
        if c == 0:
          
          dicionario['Produto'].append(int(arr.replace('M', '')))
        elif c == 1:
          
          dicionario['Etapa1: Máquina'].append(int(arr.replace('M', '')))
        elif c == 2:
          
          dicionario['Etapa1: Tempo'].append(int(arr.replace('M', '')))
        elif c == 3:
          
          dicionario['Etapa2: Máquina'].append(int(arr.replace('M', '')))
        else:
          
          dicionario['Etapa2: Tempo'].append(int(arr.replace('M', '')))
        c+=1
    t+=1


  arrays2 = np.array([string2.split('\n')])
  arrays2 = arrays2.T
  t = 0
  for array2 in arrays2:
    if t >=1:
      array2 = np.array(array2[0].split(' ')).transpose()
      c = 0
      for arr2 in array2:
        if c == 1:
          dicionario['Lucro'].append(int(arr2.replace('M', '')))
        c+=1
    t+=1


  arrays3 = np.array([string3.split('\n')])
  arrays3 = arrays3.T
  t = 0
  dicionario2 = {  "Máquina": [], "Tempo Disponível" : []}
  for array3 in arrays3:
    if t >=1:
      array3 = np.array(array3[0].split(' ')).transpose()
      c = 0
      for arr3 in array3:
        if c == 0:
          dicionario2["Máquina"].append(int(arr3.replace('M', '')))
        elif c == 1:
          dicionario2["Tempo Disponível"].append(int(arr3.replace('M', '')))
        c+=1
    t+=1
  return dicionario, dicionario2

In [ ]:
string = '''Prod Etapa 1 Etapa 2
Maq. T Maq. T
1 M1 1 M2 3
2 M4 2 M2 2
3 M1 3 M3 1
4 M4 1 M3 3
5 M1 2 M4 2
6 M2 3 M3 1'''


string2 = '''Produto Lucro ($)
P1 6
P2 5
P3 7
P4 7
P5 6
P6 5'''

string3 = '''Máquina Disp.(t)
M1 11
M2 13
M3 12
M4 10'''

dicionario, dicionario2 = stringToDict(string, string2, string3)

produtos = pd.DataFrame(dicionario)
restMaquinas = df_to_matrix(produtos, ['Etapa1: Máquina', 'Etapa2: Máquina'], ['Etapa1: Tempo', 'Etapa2: Tempo'])
disponibilidade = pd.DataFrame(dicionario2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [ ]:
matrizMaquinaProduto = restMaquinas.to_numpy()
matrizDisponibilidade = disponibilidade['Tempo Disponível'].to_numpy()
precoProdutos = produtos['Lucro'].to_numpy()
numeroDeProdutosPermitidos = 3

pcp = PCP(matrizMaquinaProduto, matrizDisponibilidade, precoProdutos, numeroDeProdutosPermitidos)
pcp.solve()
pcp.imprimirResultado()
print('-'*100)
pcp.imprimirModelo()

Resultado do PCP
Função Objetivo:  51.99999999999999
Tempo para solucionar:  24  milesegundos.
Problem solved in  9  iterações.
6 unidade do produto 2
3 unidade do produto 3
----------------------------------------------------------------------------------------------------
\ Generated by MPModelProtoExporter
\   Name             : PCP
\   Format           : Free
\   Constraints      : 11
\   Variables        : 12
\     Binary         : 6
\     Integer        : 6
\     Continuous     : 0
Maximize
 Obj: +6 x[0] +5 x[1] +7 x[2] +7 x[3] +6 x[4] +5 x[5] 
Subject to
 RestMaquinas_0: +1 x[0] +3 x[2] +2 x[4]  <= 11
 RestMaquinas_1: +3 x[0] +2 x[1] +3 x[5]  <= 13
 RestMaquinas_2: +2 x[1] +1 x[3] +2 x[4]  <= 12
 RestMaquinas_3: +1 x[2] +3 x[3] +1 x[5]  <= 10
 RestPeça_0: +1 x[0] -13 y[0]  <= 0
 RestPeça_1: +1 x[1] -13 y[1]  <= 0
 RestPeça_2: +1 x[2] -13 y[2]  <= 0
 RestPeça_3: +1 x[3] -13 y[3]  <= 0
 RestPeça_4: +1 x[4] -13 y[4]  <= 0
 RestPeça_5: +1 x[5] -13 y[5]  <= 0
 RestNumPeças3: +1 y[1] 

Questão 03

***

In [ ]:
string = '''Prod Etapa 1 Etapa 2
Maq. T Maq. T
1 M1 2 M2 3
2 M4 2 M2 3
3 M1 1 M3 1
4 M4 1 M3 1
5 M1 3 M4 2
6 M2 3 M3 2'''


string2 = '''Produto Lucro ($)
P1 6
P2 5
P3 7
P4 7
P5 6
P6 5'''

string3 = '''Máquina Disp.(t)
M1 11
M2 13
M3 12
M4 10'''

dicionario, dicionario2 = stringToDict(string, string2, string3)

produtos = pd.DataFrame(dicionario)
restMaquinas = df_to_matrix(produtos, ['Etapa1: Máquina', 'Etapa2: Máquina'], ['Etapa1: Tempo', 'Etapa2: Tempo'])
disponibilidade = pd.DataFrame(dicionario2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [ ]:
matrizMaquinaProduto = restMaquinas.to_numpy()
matrizDisponibilidade = disponibilidade['Tempo Disponível'].to_numpy()
precoProdutos = produtos['Lucro'].to_numpy()
numeroDeProdutosPermitidos = 3

pcp = PCP(matrizMaquinaProduto, matrizDisponibilidade, precoProdutos, numeroDeProdutosPermitidos)
pcp.solve()
pcp.imprimirResultado()
print('-'*100)
pcp.imprimirModelo()

Resultado do PCP
Função Objetivo:  94.00000000000001
Tempo para solucionar:  17  milesegundos.
Problem solved in  5  iterações.
6 unidade do produto 2
3 unidade do produto 3
----------------------------------------------------------------------------------------------------
\ Generated by MPModelProtoExporter
\   Name             : PCP
\   Format           : Free
\   Constraints      : 11
\   Variables        : 12
\     Binary         : 6
\     Integer        : 6
\     Continuous     : 0
Maximize
 Obj: +6 x[0] +5 x[1] +7 x[2] +7 x[3] +6 x[4] +5 x[5] 
Subject to
 RestMaquinas_0: +2 x[0] +1 x[2] +3 x[4]  <= 11
 RestMaquinas_1: +3 x[0] +3 x[1] +3 x[5]  <= 13
 RestMaquinas_2: +2 x[1] +1 x[3] +2 x[4]  <= 12
 RestMaquinas_3: +1 x[2] +1 x[3] +2 x[5]  <= 10
 RestPeça_0: +1 x[0] -13 y[0]  <= 0
 RestPeça_1: +1 x[1] -13 y[1]  <= 0
 RestPeça_2: +1 x[2] -13 y[2]  <= 0
 RestPeça_3: +1 x[3] -13 y[3]  <= 0
 RestPeça_4: +1 x[4] -13 y[4]  <= 0
 RestPeça_5: +1 x[5] -13 y[5]  <= 0
 RestNumPeças3: +1 y[1] 

Questão 04

***

In [ ]:
string = '''Prod Etapa 1 Etapa 2
Maq. T Maq. T
1 M1 1 M2 3
2 M4 2 M2 2
3 M1 3 M3 1
4 M4 1 M3 3
5 M1 2 M4 2
6 M2 3 M3 1'''


string2 = '''Produto Lucro ($)
P1 5
P2 2
P3 6
P4 3
P5 7
P6 4'''

string3 = '''Máquina Disp.(t)
M1 9
M2 12
M3 13
M4 9'''

dicionario, dicionario2 = stringToDict(string, string2, string3)

produtos = pd.DataFrame(dicionario)
restMaquinas = df_to_matrix(produtos, ['Etapa1: Máquina', 'Etapa2: Máquina'], ['Etapa1: Tempo', 'Etapa2: Tempo'])
disponibilidade = pd.DataFrame(dicionario2)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:14: FutureWarning: Using the level keyword in DataFrame and Series aggregations is deprecated and will be removed in a future version. Use groupby instead. df.sum(level=1) should use df.groupby(level=1).sum().
  


In [ ]:
matrizMaquinaProduto = restMaquinas.to_numpy()
matrizDisponibilidade = disponibilidade['Tempo Disponível'].to_numpy()
precoProdutos = produtos['Lucro'].to_numpy()
numeroDeProdutosPermitidos = 3

pcp = PCP(matrizMaquinaProduto, matrizDisponibilidade, precoProdutos, numeroDeProdutosPermitidos)
pcp.solve()
pcp.imprimirResultado()
print('-'*100)
pcp.imprimirModelo()

Resultado do PCP
Função Objetivo:  30.000000000000004
Tempo para solucionar:  10  milesegundos.
Problem solved in  7  iterações.
6 unidade do produto 2
3 unidade do produto 3
----------------------------------------------------------------------------------------------------
\ Generated by MPModelProtoExporter
\   Name             : PCP
\   Format           : Free
\   Constraints      : 11
\   Variables        : 12
\     Binary         : 6
\     Integer        : 6
\     Continuous     : 0
Maximize
 Obj: +5 x[0] +2 x[1] +6 x[2] +3 x[3] +7 x[4] +4 x[5] 
Subject to
 RestMaquinas_0: +1 x[0] +3 x[2] +2 x[4]  <= 9
 RestMaquinas_1: +3 x[0] +2 x[1] +3 x[5]  <= 12
 RestMaquinas_2: +2 x[1] +1 x[3] +2 x[4]  <= 13
 RestMaquinas_3: +1 x[2] +3 x[3] +1 x[5]  <= 9
 RestPeça_0: +1 x[0] -13 y[0]  <= 0
 RestPeça_1: +1 x[1] -13 y[1]  <= 0
 RestPeça_2: +1 x[2] -13 y[2]  <= 0
 RestPeça_3: +1 x[3] -13 y[3]  <= 0
 RestPeça_4: +1 x[4] -13 y[4]  <= 0
 RestPeça_5: +1 x[5] -13 y[5]  <= 0
 RestNumPeças3: +1 y[1] +